In [ ]:
import pandas as pd

In [ ]:
#%run ../../SSB_Spark_tools/data_processing/Utils.py

In [ ]:
%run ../ssb_sparktools/editing/editing.py

In [ ]:
%run ../ssb_sparktools/quality/quality.py

In [ ]:
%run ../ssb_sparktools/processing/processing.py

In [ ]:
skattRaw = spark.read.path('/produkt/skatt/person/temp/tverrsnitt_skattemelding')

In [ ]:
skatt_dict = unpack_parquet(skattRaw, rootdf=True, rootvar=True)

In [ ]:
#Kjører missing_df for alle skatteobjekter og samler resultatet i en dictionary
skatt_missing = {}
for k in skatt_dict.keys():
    skatt_missing[k] = missing_df(skatt_dict[k])

In [ ]:
#Eksempel på resultat fra missing_df
skatt_missing['rootdf']

In [ ]:
#Enda et eksempel på resultat fra missing_df
skatt_missing['fondskonto']

In [ ]:
#Kjører korrigering på bolske verdier på et gitt datasett. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_bool, missing_boolcount = missing_correction_bool(skatt_dict['fondskonto'])
for k, v in missing_boolcount.items():
    print(k, v)

In [ ]:
#Eksempel før korrigering
skatt_dict['fondskonto'][['identifikator', 'overfoertFraBarn']].toPandas()

In [ ]:
#Eksempel etter korrrigering
df_corrected_bool[['identifikator', 'overfoertFraBarn']].toPandas()

In [ ]:
#Eksempel før korrigering der overfoertFraBarn skal unntas korrigering
skatt_dict['konto'].select('identifikator', 'erBoligsparekontoIAnnenEOeSStat', 'overfoertFraBarn').toPandas()

In [ ]:
#Kjører korrigering på bolske verdier på et gitt datasett med unntak for 1 variabel. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_bool, missing_boolcount = missing_correction_bool(skatt_dict['konto'], exception_for=['overfoertFraBarn'])

In [ ]:
#Eksempel før korrigering der overfoertFraBarn skal unntas korrigering
df_corrected_bool.select('identifikator', 'erBoligsparekontoIAnnenEOeSStat', 'overfoertFraBarn').toPandas()

In [ ]:
#Kjører korrigering på numeriske verdier på et gitt datasett. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_number, missing_numbcount = missing_correction_number(skatt_dict['fondskonto'])
for k, v in missing_numbcount.items():
    print(k, v)

In [ ]:
#Eksempel før korrigering
skatt_dict['fondskonto']\
                [['identifikator', 'skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel','skattepliktigTapVedRealisasjonAvOgUttakFraRentedel',\
                 'verdiFoerVerdsettingsrabattForAksjeOgAksjefond', 'skattefrittUttak', 'benyttetSkjermingsfradrag']]\
                .toPandas()

In [ ]:
#Eksempel etter korrigering
df_corrected_number\
                [['identifikator', 'skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel','skattepliktigTapVedRealisasjonAvOgUttakFraRentedel',\
                 'verdiFoerVerdsettingsrabattForAksjeOgAksjefond', 'skattefrittUttak', 'benyttetSkjermingsfradrag']]\
                .toPandas()

In [ ]:
#Kjører korrigering på numeriske verdier på et gitt datasett. Datasett korrigeres og returneres sammen med log over korrigeringer
df_corrected_number, missing_numbcount = missing_correction_number(skatt_dict['fondskonto'], exception_for=['skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel'])

In [ ]:
#Eksempel etter korrigering det skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel er unntatt korrigering
df_corrected_number\
                [['identifikator', 'skattepliktigTapVedRealisasjonAvOgUttakFraAksjedel','skattepliktigTapVedRealisasjonAvOgUttakFraRentedel',\
                 'verdiFoerVerdsettingsrabattForAksjeOgAksjefond', 'skattefrittUttak', 'benyttetSkjermingsfradrag']]\
                .toPandas()

In [ ]:
#Eksempel som kjører korrigering av numeriske og bolske verdier for alle skatteobjekt og samler loggene i et eget datasett
corrected_missing_log = pd.DataFrame(columns=['skatteobjekt', 'datatype', 'variabel', 'antall_korrigeringer'])

for k in skatt_dict.keys():
    
    df, miss_bool = missing_correction_bool(skatt_dict[k])
    for l, m in miss_bool.items():
        pdrow = pd.DataFrame([[k, 'Boolske', l, m]], columns=['skatteobjekt', 'datatype', 'variabel', 'antall_korrigeringer'])
        corrected_missing_log = corrected_missing_log.append(pdrow)
        

    df, miss_number = missing_correction_number(df)
    for l, m in miss_number.items():
        pdrow = pd.DataFrame([[k, 'Number', l, m]], columns=['skatteobjekt', 'datatype', 'variabel', 'antall_korrigeringer'])
        corrected_missing_log.append(pdrow)
        
corrected_missing_log = corrected_missing_log.reset_index(drop=True)

In [ ]:
#Log av datesett
corrected_missing_log